# Artificial and Computational Intelligence Assignment 1

## Problem solving by Uninformed & Informed Search

List only the BITS (Name) of active contributors in this assignment:
1. Ritesh Ranjan
2. Shashank Singh
3. Divyanshu Singh
4. Adapa Hemachand

Things to follow
1.	Use appropriate data structures to represent the graph and the path using python libraries
2.	Provide proper documentation
3.	Find the path and print it

Coding begins here

### 1.	Define the environment in the following block

PEAS (Performance measure, Environment, Actuator, Sensor):

1. Performance measure: The performance measure for the rescue robot is to find the optimal path from the start (S) to the goal (G) with the minimum cost. The cost includes the path cost of +3 for each transition, +5 for moving towards the fire cell, and +1 for moving towards the bush plant cell. The robot's objective is to minimize the total cost incurred during the pathfinding process.

2. Environment: The environment is represented as a 2D grid, where each cell can be either empty ('O'), a wall ('#'), fire ('F'), bush plant ('B'), the start ('S'), or the goal ('G'). The robot has to navigate through this grid while avoiding obstacles like walls, fire, and bushes.

3. Actuator: The actuator for the rescue robot allows it to move in four directions: north, south, west, and east. It can move to neighboring cells in the grid based on its current position.

4. Sensor: The robot's sensors provide information about the grid's current state, including the presence of walls, fire, bushes, and the goal. The sensors also detect the type of cells surrounding the robot, helping it make decisions based on the information it receives.

In [1]:
#Code Block : Set Initial State (Must handle dynamic inputs)

# Global variables to track time and space complexity
expansions = 0
max_queue_size = 0

def find_start_and_goal_indices(grid):
    # Find the indices of the start and goal positions in the grid
    start_index = None
    goal_index = None

    for row_idx, row in enumerate(grid):
        for col_idx, cell in enumerate(row):
            if cell == 'S':
                start_index = (row_idx, col_idx)
            elif cell == 'G':
                goal_index = (row_idx, col_idx)

            # If both 'S' and 'G' are found, we can stop the loop early
            if start_index and goal_index:
                break

    return start_index, goal_index


In [2]:
#Code Block : Set the matrix for transition & cost (as relevant for the given problem)

# Function to check if a cell is valid and not a wall
def is_valid_cell(cell, grid):
    x, y = cell
    return 0 <= x < len(grid) and 0 <= y < len(grid[0]) and grid[x][y] != '#'

In [3]:
#Code Block : Write function to design the Transition Model/Successor function. Ideally this would be called while search algorithms are implemented

# Helper function to calculate Manhattan distance between two points
def manhattan_distance(start, goal):
    return abs(start[0] - goal[0]) + abs(start[1] - goal[1])

def calculate_space_complexity():
    # Calculate the maximum size of the queue during the search
    return max_queue_size


### 2.	Definition of Algorithm 1 (Iterative Deepening A*)

In [4]:
#Code Block : Function for algorithm 1 implementation

# Function to find the optimal path using IDA*
def ida_star_search(grid, start, goal, path, path_costs):
    global expansions, max_queue_size
    expansions = 0
    
    def dfs(node, g, threshold):
        global expansions
        expansions += 1
        f = g + manhattan_distance(node, goal)
        if f > threshold:
            return f
        if node == goal:
            path.append(node)
            return -1  # Path found
        min_cost = float('inf')
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = node[0] + dx, node[1] + dy
            new_node = (nx, ny)
            if is_valid_cell(new_node, grid) and new_node not in visited:
                visited.add(new_node)
                cost = 3  # Cost of a transition
                if grid[new_node[0]][new_node[1]] == 'F':
                    cost += 5  # Penalty for moving towards fire
                elif grid[new_node[0]][new_node[1]] == 'B':
                    cost += 1  # Additional cost for moving towards bush plant
                new_g = g + cost
                t = dfs(new_node, new_g, threshold)
                if t == -1:
                    path.append(node)
                    path_costs.append(cost)
                    return -1
                if t < min_cost:
                    min_cost = t
                visited.remove(new_node)
        return min_cost

    threshold = manhattan_distance(start, goal)
    while True:
        # Reset queue size for each depth limit iteration
        max_queue_size = 0
        visited = {start}
        t = dfs(start, 0, threshold)
        if t == -1:
            return threshold
        if t == float('inf'):
            return None
        threshold = t


### 3.	Definition of Algorithm 2 (Hill Climbing)

In [5]:
#Code Block : Function for algorithm 2 implementation

# Hill Climbing Search
def hill_climbing_search(grid, start, goal, path, path_costs):
    global expansions
    expansions = 0

    def calculate_step_cost(current_node, next_node):
        x, y = next_node
        cost = 3  # Cost of a transition
        if grid[x][y] == 'F':
            cost += 5  # Penalty for moving towards fire
        elif grid[x][y] == 'B':
            cost += 1  # Additional cost for moving towards bush plant
        return cost

    current_node = start
    path.append(current_node)

    while current_node != goal:
        global max_queue_size
        expansions += 1
        neighbors = []
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = current_node[0] + dx, current_node[1] + dy
            new_node = (nx, ny)
            if is_valid_cell(new_node, grid) and new_node not in path:
                neighbors.append(new_node)

        if not neighbors:
            # If no valid neighbors are found, we are stuck, and the search fails
            return None

        # Pick the neighbor with the lowest cost
        min_cost = manhattan_distance(neighbors[0], goal) + calculate_step_cost(current_node, neighbors[0])
        next_node = neighbors[0]
        for neighbor in neighbors:
            neighbor_cost = manhattan_distance(neighbor, goal) + calculate_step_cost(current_node, neighbor)
            if neighbor_cost < min_cost:
                min_cost = neighbor_cost
                next_node = neighbor

        # Move to the next node and update the current cost
        current_node = next_node
        path.append(current_node)
        path_costs.append(calculate_step_cost(current_node, next_node))

    return sum(path_costs)


### DYNAMIC INPUT

IMPORTANT : Dynamic Input must be got in this section. Display the possible states to choose from:
This is applicable for all the relevent problems as mentioned in the question.

In [6]:
#Code Block : Function & call to get inputs (start/end state)
def print_environment(grid):
    # Function to print the environment grid
    for row in grid:
        print(' '.join(row))
        
def replace_cell(grid, cell_type, new_cell):
    # Function to replace all occurrences of a specific cell type with a new cell in the grid
    for row_idx, row in enumerate(grid):
        for col_idx, cell in enumerate(row):
            if cell == cell_type:
                grid[row_idx][col_idx] = new_cell
        
environment = [
        ['#','#','#','#','#','S','#','#','#','#','#','#','#'],
        ['#','B','O','O','O','O','#','O','O','O','O','F','#'],
        ['#','O','#','#','#','O','#','#','#','#','#','O','#'],
        ['#','O','#','O','#','O','O','O','O','B','#','O','#'],
        ['#','O','#','O','#','O','#','#','#','O','#','O','#'],
        ['#','O','#','O','O','O','#','F','O','O','O','O','#'],
        ['#','O','#','#','#','O','#','#','#','#','#','O','#'],
        ['#','O','O','F','#','O','#','O','O','O','O','O','#'],
        ['#','O','#','#','#','O','#','O','#','#','#','#','#'],
        ['#','O','#','B','O','O','O','O','O','O','O','B','#'],
        ['#','O','#','O','#','#','#','#','#','O','#','O','#'],
        ['#','O','#','O','O','O','O','O','#','O','#','O','#'],
        ['#','#','#','O','#','#','#','O','#','O','#','O','#'],
        ['#','O','O','O','#','O','O','O','O','O','#','O','#'],
        ['#','#','#','#','#','#','#','#','#','G','#','#','#'],
    ]

# Print the original environment
print("Original Environment:")
print_environment(environment)

# Explain the meaning of different cell types
print("\nCell Types:")
print("# - Blocked (Wall)")
print("O - Empty cell (Moveable)")
print("S - Start")
print("G - Goal state")
print("B - Grass bush with extra cost +1")
print("F - Fire with extra cost +5\n")

start, goal = find_start_and_goal_indices(environment)
print("Starting position:", start)
print("Goal position:", goal)

# Ask the user if they want to change the start and goal positions
change_start_goal = input("\nDo you want to change the start and goal positions? (yes/no): ").lower()

if change_start_goal == 'yes':
    # Replace the previous start and goal positions with '#'
    replace_cell(environment, 'S', '#')
    replace_cell(environment, 'G', '#')

    # Get new start and goal positions from the user
    new_start_input = input("Enter the row and column indices for the new start position (e.g., '3 5'): ").split()
    new_goal_input = input("Enter the row and column indices for the new goal position (e.g., '14 10'): ").split()

    # Convert user inputs to integers
    new_start_row, new_start_col = map(int, new_start_input)
    new_goal_row, new_goal_col = map(int, new_goal_input)

    # Set the new start and goal positions in the grid
    environment[new_start_row][new_start_col] = 'S'
    environment[new_goal_row][new_goal_col] = 'G'

    # Print the updated environment
    print("\nUpdated Environment with new start and goal positions:")
    print_environment(environment)

start, goal = find_start_and_goal_indices(environment)
print("\nStarting position:", start)
print("Goal position:", goal)

Original Environment:
# # # # # S # # # # # # #
# B O O O O # O O O O F #
# O # # # O # # # # # O #
# O # O # O O O O B # O #
# O # O # O # # # O # O #
# O # O O O # F O O O O #
# O # # # O # # # # # O #
# O O F # O # O O O O O #
# O # # # O # O # # # # #
# O # B O O O O O O O B #
# O # O # # # # # O # O #
# O # O O O O O # O # O #
# # # O # # # O # O # O #
# O O O # O O O O O # O #
# # # # # # # # # G # # #

Cell Types:
# - Blocked (Wall)
O - Empty cell (Moveable)
S - Start
G - Goal state
B - Grass bush with extra cost +1
F - Fire with extra cost +5

Starting position: (0, 5)
Goal position: (14, 9)

Do you want to change the start and goal positions? (yes/no): 

Starting position: (0, 5)
Goal position: (14, 9)


### 4.	Calling the search algorithms
(For bidirectional search in below sections first part can be used as per Hint provided. Under second section other combinations as per Hint or your choice of 2 algorithms can be called .As an analyst suggest suitable approximation in the comparitive analysis section)

In [7]:
#Invoke algorithm 1 (Should Print the solution, path, cost etc., (As mentioned in the problem))
#IDA*
visited = set()
path_ida, path_costs_ida = [], []
path_cost_ida = ida_star_search(environment, start, goal, path_ida, path_costs_ida)

if path_cost_ida is not None:
    print(f"IDA* Optimal path cost: {path_cost_ida}")
    print("IDA* Optimal path sequence:", path_ida[::-1])  # Reverse the path sequence for correct order
    print("IDA* Path costs:", path_costs_ida[::-1])  # Reverse the path costs for correct order
else:
    print("IDA* No path found!")

IDA* Optimal path cost: 54
IDA* Optimal path sequence: [(0, 5), (1, 5), (2, 5), (3, 5), (4, 5), (5, 5), (6, 5), (7, 5), (8, 5), (9, 5), (9, 6), (9, 7), (9, 8), (9, 9), (10, 9), (11, 9), (12, 9), (13, 9), (14, 9)]
IDA* Path costs: [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [8]:
#Invoke algorithm 2 (Should Print the solution, path, cost etc., (As mentioned in the problem))
# Hill Climbing Search
path_hill_climbing, path_costs_hill_climbing = [], []
path_cost_hill_climbing = hill_climbing_search(environment, start, goal, path_hill_climbing, path_costs_hill_climbing)

if path_cost_hill_climbing is not None:
    print("\nHill Climbing Optimal path cost:", path_cost_hill_climbing)
    print("Hill Climbing Optimal path sequence:", path_hill_climbing)
    print("Hill Climbing Path costs:", path_costs_hill_climbing)
else:
    print("Hill Climbing: No path found!")


Hill Climbing Optimal path cost: 54
Hill Climbing Optimal path sequence: [(0, 5), (1, 5), (2, 5), (3, 5), (4, 5), (5, 5), (6, 5), (7, 5), (8, 5), (9, 5), (9, 6), (9, 7), (9, 8), (9, 9), (10, 9), (11, 9), (12, 9), (13, 9), (14, 9)]
Hill Climbing Path costs: [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


### 5.	Comparitive Analysis

In [9]:
#Code Block : Print the Time & Space complexity of algorithm 1
# Calculate space complexity for Iterative Deepening A* Algorithm
space_complexity = calculate_space_complexity()
print("Space Complexity:", space_complexity)
print("Time Complexity:", expansions)

Space Complexity: 0
Time Complexity: 18


In [10]:
#Code Block : Print the Time & Space complexity of algorithm 2

# Calculate space complexity for Hill Climbing Algorithm
space_complexity_hill_climbing = calculate_space_complexity()
print("Space Complexity (Hill Climbing):", space_complexity_hill_climbing)
print("Time Complexity (Hill Climbing):", expansions)

Space Complexity (Hill Climbing): 0
Time Complexity (Hill Climbing): 18


### 6.	Provide your comparitive analysis or findings in no more than 3 lines in below section

Comparison : _______________________________________________

1. Based on the characteristics of the environment and the requirements for finding the optimal path, IDA* is likely the better choice. It guarantees optimality and can effectively handle complex search spaces with obstacles. However, if you are looking for a faster but potentially suboptimal solution, Hill Climbing could be considered.
2. In this environment, Hill Climbing may struggle to find the optimal path due to obstacles and complex structures. It could get stuck in local optima or fail to find a solution altogether.
________________________________________________________

_________________________________________________________